In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Processing hdf5 data

This notebook provides examples for processing hdf5 data

In [3]:
from e11 import H5Data, add_column_index
from e11.process import vrange, statistics

In [4]:
# read file
import os 
fil = os.path.join(os.getcwd(), 'example_data', 'array_data.h5')
h5 = H5Data(fil)

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 599.63it/s]


In [5]:
h5.log

,ACQUIRE,DATETIME,END,ERROR,LOOP,OVERRIDE,START,VAR:PAUSE,ELAPSED
squid,,,,,,,,,
1,10.036975,2017-12-01 15:41:22,3.594988e+09,0,1,0,3.594988e+09,300,00:00:00
2,10.021719,2017-12-01 15:41:33,3.594988e+09,0,1,0,3.594988e+09,600,00:00:11
3,10.034403,2017-12-01 15:41:45,3.594988e+09,0,1,0,3.594988e+09,900,00:00:23
4,10.022052,2017-12-01 15:41:55,3.594988e+09,0,2,0,3.594988e+09,300,00:00:33
5,10.027435,2017-12-01 15:42:06,3.594988e+09,0,2,0,3.594988e+09,600,00:00:44
6,10.027131,2017-12-01 15:42:18,3.594988e+09,0,2,0,3.594988e+09,900,00:00:56


## Vrange

Here, we are applying the vrange function to measure the vertical range of array data.

In [6]:
vr, info = vrange(h5, 'OSC_0', info=True)
vr.head()

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 206.77sq/s]


vrange
squid repeat          
1     0       1.033028
      1       1.012191
      2       1.032403
      3       1.070328
      4       1.062143

In [7]:
# information about the processing
info

{'dataset': 'OSC_0',
 'datetime': '2018-01-10 13:00:04',
 'process': 'process.vrange()',
 'squids': array([1, 2, 3, 4, 5, 6])}

And use the statistics function to find the average vrange per squid.

In [8]:
av = statistics(vr)
av.head()

vrange                              
       count       err      mean       std
squid                                     
1         25  0.003350  1.038482  0.016752
2         25  0.002847  1.039364  0.014236
3         26  0.001758  1.043457  0.008966
4         25  0.002988  1.044290  0.014941
5         25  0.003784  1.034683  0.018921

We can see from h5.log that there were repeat measurements with the same VAR conditions.  We can group these together and find the statistics for unique VAR values.

In [9]:
# map measurements to var values
df = h5.var.join(vr)
# evaluate the statistics when grouped by var values
st = statistics(df, groupby=h5.var.columns, mode='full')
st.head()

vrange                                                              \
       count       err       max      mean    median       min     range   
PAUSE                                                                      
300       50  0.002260  1.083854  1.041386  1.040635  1.000539  0.083316   
600       50  0.002367  1.064080  1.037024  1.042134  0.997321  0.066759   
900       51  0.001702  1.069203  1.040724  1.041619  1.012878  0.056325   

                 
            std  
PAUSE            
300    0.015981  
600    0.016739  
900    0.012155